In [14]:
import subprocess
subprocess.check_output(["start","./onbpr4"],shell=True)

b''

In [132]:
import os
import numpy as np
import os
import numpy as np


def trace_inner_product(a, b):
           return not (a == 0 or b == 0 or a == b)
class FileReader:
    def __init__(self, n, k, m, code_type, trained):
        self.mycodetype = code_type
        self.N = n
        self.K = k
        self.M = m
        self.G_rows = n + k
        self.mTrained = trained
        self.maxDv = 0
        self.maxDc = 0
        self.dv = []
        self.dc = []
        self.Nvk = []
        self.Mck = []
        self.Nv = []
        self.Mc = []
        self.checkVal = []
        self.varVal = []
        self.G = []
        self.weights_cn = []
        self.weights_ri = []
        self.weights_llr = []
        self.weights_vn = []
        self.trained_iter = 0
        self.read_H()
        self.read_G()
        if trained:
            self.load_cn_weights()
            self.load_llr_weights()
            self.load_vn_weights()
            self.load_ri_weights()
            





    def read_H(self):
        code_type_string = self.code_type_string()
        filename = f"./PCMs/{code_type_string}_{self.N}_{self.K}/{code_type_string}_{self.N}_{self.K}_H_{self.M}.alist"

        checkValues = []
        VariableValues = []

        with open(filename, 'r') as matrix_file:
            # First line: n k
            n, m = map(int, matrix_file.readline().split())
            if n != self.N:
                raise RuntimeError("read_H: file-specified N not as expected")
            if m != self.M:
                raise RuntimeError("read_H: file-specified M not as expected")

            # Second line: max dv and dc
            maxDv, maxDc = map(int, matrix_file.readline().split())

            # Third line: dv degrees
            dv = list(map(int, matrix_file.readline().split()))

            # Fourth line: dc degrees
            dc = list(map(int, matrix_file.readline().split()))

            Nvk = [[] for _ in range(n)]
            Mck = [[] for _ in range(m)]

            # Fifth to n+5-th line: dv neighbors
            Nv = []
            for i in range(n):
                line = list(map(int, matrix_file.readline().split()))
                Nv.append([0] * dv[i])
                for j in range(dv[i]):
                    Nv[i][j] = line[j] - 1
                    Mck[Nv[i][j]].append(j)

            # n+6-th to n+6+m-th line: dc neighbors
            Mc = []
            for i in range(m):
                line = list(map(int, matrix_file.readline().split()))
                Mc.append([0] * dc[i])
                
                for j in range(dc[i]):
                    Mc[i][j] = line[j] - 1
                    
                    Nvk[Mc[i][j]].append(j)

            # n+6+m+1-th to n+6+m+n-th line: value of each row
            for i in range(m):
                line = list(map(int, matrix_file.readline().split()))
                #while line[-1] == 0:
                  # line.pop()
                checkValues.append(line)
                

            # Last lines: value of each column
            for i in range(n):
                line = list(map(int, matrix_file.readline().split()))
                VariableValues.append(line)
        self.maxDv = maxDv
        self.maxDc = maxDc
        self.dv = dv
        self.dc = dc
        self.Nvk = Nvk
        self.Mck = Mck
        self.Nv = Nv
        self.Mc = Mc
        self.checkVal = checkValues
        self.varVal = VariableValues
        
        print("Length of dv:", len(self.dv))
        print("Length of dc:", len(self.dc))
        print("Length of Nvk:", len(self.Nvk))
        print("Length of Mck:", len(self.Mck))
        print("Length of Nv:", len(self.Nv))
        print("Length of Mc:", len(self.Mc))
        print("Length of checkVal:", len(self.checkVal))
        print("Length of varVal:", len(self.varVal))
        print(len(self.Nvk))
        
    def save_values_to_file(self):
        with open("values.txt", "w") as file:
            file.write("maxDv: " + str(self.maxDv) + "\n")
            file.write("maxDc: " + str(self.maxDc) + "\n")
            file.write("dv: " + str(self.dv) + "\n")
            file.write("dc: " + str(self.dc) + "\n")
            file.write("Nvk: " + str(self.Nvk) + "\n")
            file.write("Mck: " + str(self.Mck) + "\n")
            file.write("Nv: " + str(self.Nv) + "\n")
            file.write("Mc: " + str(self.Mc) + "\n")
            file.write("checkVal: " + str(self.checkVal) + "\n")
            file.write("varVal: " + str(self.varVal) + "\n")
        
        

    
    def read_Mck_from_file(self, filename):
        with open(filename, 'r') as file:
             lines = file.readlines()
        # Assuming each line in Mck.txt contains a list of integers separated by whitespace
             Mck_from_file = [list(map(int, line.strip().split())) for line in lines]
        return Mck_from_file

    def compare_Mck_with_file(self, filename):
        Mck_from_file = self.read_Mck_from_file(filename)
    
        if len(Mck_from_file) != len(self.Mck):
           print("Length of Mck does not match the length of Mck from file.")
           return False
    
        for i in range(len(self.Mck)):
            if len(Mck_from_file[i]) != len(self.Mck[i]):
                print(f"Length of Mck[{i}] does not match the length of Mck[{i}] from file.")
                return False
        
            for j in range(len(self.Mck[i])):
               if Mck_from_file[i][j] != self.Mck[i][j]:
                   print(f"Mismatch found in Mck[{i}][{j}]: Expected {Mck_from_file[i][j]}, Got {self.Mck[i][j]}")
                   return False
            
        print("Mck matches the values from file.")
        return True
    
    def read_Nvk_from_file(self, filename):
        with open(filename, 'r') as file:
             lines = file.readlines()
        # Assuming each line in Mck.txt contains a list of integers separated by whitespace
             Nvk_from_file = [list(map(int, line.strip().split())) for line in lines]
        return Nvk_from_file
    
    
    def compare_Nvk_with_file(self, filename):
        Nvk_from_file = self.read_Nvk_from_file(filename)
    
        if len(Nvk_from_file) != len(self.Nvk):
           print("Length of Nvk does not match the length of Nvkk from file.")
           return False
    
        for i in range(len(self.Nvk)):
            if len(Nvk_from_file[i]) != len(self.Nvk[i]):
                print(f"Length of Nvk[{i}] does not match the length of Nvk[{i}] from file.")
                return False
        
            for j in range(len(self.Nvk[i])):
               if Nvk_from_file[i][j] != self.Nvk[i][j]:
                   print(f"Mismatch found in Nvk[{i}][{j}]: Expected {Nvk_from_file[i][j]}, Got {self.Nvk[i][j]}")
                   return False
            
        print("Nvk matches the values from file.")
        return True
    
    def read_Mc_from_file(self, filename):
        with open(filename, 'r') as file:
             lines = file.readlines()
        # Assuming each line in Mck.txt contains a list of integers separated by whitespace
             Mc_from_file = [list(map(int, line.strip().split())) for line in lines]
        return Mc_from_file
    
    def compare_Mc_with_file(self, filename):
        Mc_from_file = self.read_Mc_from_file(filename)
    
        if len(Mc_from_file) != len(self.Mc):
           print("Length of Mc does not match the length of Nvkk from file.")
           return False
    
        for i in range(len(self.Mc)):
            if len(Mc_from_file[i]) != len(self.Mc[i]):
                print(f"Length of Mc[{i}] does not match the length of Mc[{i}] from file.")
                return False
        
            for j in range(len(self.Mc[i])):
               if Mc_from_file[i][j] != self.Mc[i][j]:
                   print(f"Mismatch found in Mc[{i}][{j}]: Expected {Mc_from_file[i][j]}, Got {self.Mc[i][j]}")
                   return False
            
        print("Mc matches the values from file.")
        return True
    
    def read_Nv_from_file(self, filename):
        with open(filename, 'r') as file:
             lines = file.readlines()
        # Assuming each line in Mck.txt contains a list of integers separated by whitespace
             Nv_from_file = [list(map(int, line.strip().split())) for line in lines]
        return Nv_from_file
    
    def compare_Nv_with_file(self, filename):
        Nv_from_file = self.read_Nv_from_file(filename)
    
        if len(Nv_from_file) != len(self.Nv):
           print("Length of Mc does not match the length of Nvkk from file.")
           return False
    
        for i in range(len(self.Nv)):
            if len(Nv_from_file[i]) != len(self.Nv[i]):
                print(f"Length of Nv[{i}] does not match the length of Nv[{i}] from file.")
                return False
        
            for j in range(len(self.Mc[i])):
               if Nv_from_file[i][j] != self.Nv[i][j]:
                   print(f"Mismatch found in Nv[{i}][{j}]: Expected {Nv_from_file[i][j]}, Got {self.Nv[i][j]}")
                   return False
            
        print("Nv matches the values from file.")
        return True
    
    def read_checkVal_from_file(self, filename):
        with open(filename, 'r') as file:
             lines = file.readlines()
        # Assuming each line in Mck.txt contains a list of integers separated by whitespace
             checkVal_from_file = [list(map(int, line.strip().split())) for line in lines]
        return checkVal_from_file
    
    def compare_checkVal_with_file(self, filename):
        checkVal_from_file = self.read_checkVal_from_file(filename)
    
        if len(checkVal_from_file) != len(self.checkVal):
           print("Length of checkVal does not match the length of checkVal from file.")
           return False
    
        for i in range(len(self.checkVal)):
            if len(checkVal_from_file[i]) != len(self.checkVal[i]):
                print(f"Length of checkVal[{i}] does not match the length of checkVal[{i}] from file.")
                return False
        
            for j in range(len(self.checkVal[i])):
               if checkVal_from_file[i][j] != self.checkVal[i][j]:
                   print(f"Mismatch found in checkVal[{i}][{j}]: Expected {checkVal_from_file[i][j]}, Got {self.checkVal[i][j]}")
                   return False
            
        print("checkVal matches the values from file.")
        return True
    
    def read_varVal_from_file(self, filename):
        with open(filename, 'r') as file:
             lines = file.readlines()
        # Assuming each line in Mck.txt contains a list of integers separated by whitespace
             varVal_from_file = [list(map(int, line.strip().split())) for line in lines]
        return varVal_from_file
    
    
    def compare_varVal_with_file(self, filename):
        varVal_from_file = self.read_varVal_from_file(filename)
    
        if len(varVal_from_file) != len(self.varVal):
           print("Length of varVal does not match the length of varVal from file.")
           return False
    
        for i in range(len(self.varVal)):
            if len(varVal_from_file[i]) != len(self.varVal[i]):
                print(f"Length of varVal[{i}] does not match the length of varVal[{i}] from file.")
                return False
        
            for j in range(len(self.varVal[i])):
               if varVal_from_file[i][j] != self.varVal[i][j]:
                   print(f"Mismatch found in varVal[{i}][{j}]: Expected {varVal_from_file[i][j]}, Got {self.varVal[i][j]}")
                   return False
            
        print("varVal matches the values from file.")
        return True
    
    
    
    def read_G(self):
        code_type_string = self.code_type_string()
        filename = f"./PCMs/{code_type_string}_{self.N}_{self.K}/{code_type_string}_{self.N}_{self.K}_G.txt"
        with open(filename, 'r') as matrix_file:
            for _ in range(self.G_rows):
                row = list(map(int, matrix_file.readline().split()))
                self.G.append(row)
        print(self.G)

    def load_cn_weights(self):
        path = self.construct_weights_path("weight_cn.txt")
        weight_cn = []
        with open(path, 'r') as weights_file:
            dec_iter = int(weights_file.readline())
            self.trained_iter = dec_iter
            n, m = map(int, weights_file.readline().split())
            assert n == self.N, "load_cn_weights: file-specified N not as expected"
            assert m == self.M, "load_cn_weights: file-specified M not as expected"
            for _ in range(dec_iter):
                weight_cn_tmp = []
                for _ in range(m):
                    weight_cn_row = list(map(float, weights_file.readline().split()))
                    weight_cn_tmp.append(weight_cn_row)
                weight_cn.append(weight_cn_tmp)
        self.weights_cn = weight_cn
        
    def load_ri_weights(self):
        path = self.construct_weights_path("weight_ri.txt")
        weight_ri = []
        with open(path, 'r') as weights_file:
            dec_iter = int(weights_file.readline())
            self.trained_iter = dec_iter
            n, m = map(int, weights_file.readline().split())
            assert n == self.N, "load_ri_weights: file-specified N not as expected"
            assert m == self.M, "load_ri_weights: file-specified M not as expected"
            for _ in range(dec_iter):
                weight_ri_tmp = []
                for _ in range(m):
                    weight_ri_row = list(map(float, weights_file.readline().split()))
                    weight_ri_tmp.append(weight_ri_row)
                weight_ri.append(weight_ri_tmp)
        self.weights_ri = weight_ri

    def load_llr_weights(self):
        path = self.construct_weights_path("weight_llr.txt")
        weight_llr = []
        with open(path, 'r') as weights_file:
            dec_iter = int(weights_file.readline())
            for _ in range(dec_iter):
                weight_llr.append(list(map(float, weights_file.readline().split())))
        self.weights_llr = weight_llr

    def load_vn_weights(self):
        path = self.construct_weights_path("weight_vn.txt")
        weight_vn = []
        with open(path, 'r') as weights_file:
            dec_iter = int(weights_file.readline())
            n, m = map(int, weights_file.readline().split())
            assert n == self.N, "load_vn_weights: file-specified N not as expected"
            assert m == self.M, "load_vn_weights: file-specified M not as expected"
            for _ in range(dec_iter):
                weight_vn_tmp = []
                for _ in range(n):
                    weight_vn_row = list(map(float, weights_file.readline().split()))
                    weight_vn_tmp.append(weight_vn_row)
                weight_vn.append(weight_vn_tmp)
        self.weights_vn = weight_vn

    def construct_weights_path(self, filename):
        code_type_string = self.code_type_string()
        directory_name_builder = f"{code_type_string}_{self.N}_{self.K}_{self.M}"
        path = os.path.join("training_results", directory_name_builder, filename)
        return path

    def code_type_string(self):
        if self.mycodetype == 'GB':
            return "GB"
        elif self.mycodetype == 'HP':
            return "HP"
        elif self.mycodetype == 'toric':
            return "toric"
        else:
            raise ValueError("Unimplemented codetype")
    
    #about this, the matrices are identical, probably there is an error in the code, but with c++ works so no problem?\
        
    
       
    def check_symplectic(self):
        vec1 = [0] * self.N
        vec2 = [0] * self.N
        
        for rowid in range(self.M):
                 for iii in range(self.N):
                     print("ciao")
                     vec1[iii] = 0
                 for j in range(len(self.Mc[rowid])):
                     vec1[self.Mc[rowid][j]] = self.checkVal[rowid][j]
                     print(vec1[j], end=" ")

                 for i in range(self.M):
                    for iii in range(self.N):
                       vec2[iii] = 0
                    for j in range(len(self.Mc[i])):
                       vec2[self.Mc[i][j]] = self.checkVal[i][j]
                 syn_check = False
                 for j in range(self.N):
                    syn_check = not (vec1[j] == 0 or vec2[j] == 0 or vec1[j] == vec2[j])
                    syn_check = not syn_check if syn_check else syn_check
                 if syn_check:
                     raise RuntimeError("check_symplectic: syn_check % 2 != 0")
                 
                 for i in range(self.G_rows):
                     syn_check = False
                     for j in range(self.N):
                         syn_check = not (vec1[j] == 0 or self.G[i][j] == 0 or vec1[j] == self.G[i][j])
                         syn_check = not syn_check if syn_check else syn_check
                     if syn_check:
                         raise RuntimeError("check_symplectic / GH^T: syn_check % 2 != 0")
                #syn_check = False
                #for j in range(self.N):
                #   syn_check = self.trace_inner_product(vec1[j], vec2[j]) != 0
                #   if syn_check:
                #        raise ValueError("check_symplectic: syn_check % 2 != 0")

          #  for i in range(self.G_rows):
           #     syn_check = False
            #    for j in range(self.N):
             #       syn_check = self.trace_inner_product(vec1[j], self.G[i][j]) != 0
              #      if syn_check:
               #         raise ValueError("check_symplectic / GH^T: syn_check % 2 != 0")

        print("Check symplectic ok")
        return True
        
    
    




In [133]:
import math
import random
import numpy as np

class StabilizerCodes:
    def __init__(self, n, k, m, code_type, fr, trained):
        self.mycodetype = code_type
        self.N = n
        self.K = k
        self.M = m
        self.trained_iter = 0  # Define trained_iter here
        self.G_rows = self.N + self.K
        self.mTrained = trained
        self.dc = fr.dc
        self.dv = fr.dv
        self.maxDc = fr.maxDc
        self.maxDv = fr.maxDv
        self.Nv = fr.Nv
        self.Mc = fr.Mc
        self.checkVal = fr.checkVal
        self.varVal = fr.varVal
        self.Nvk = fr.Nvk
        self.Mck = fr.Mck
        self.G = fr.G
        if trained:
            self.weights_cn = fr.weights_cn
            self.weights_vn = fr.weights_vn
            self.weights_llr = fr.weights_llr

    def decode(self, L, epsilon):
        if not self.errorString:
            return [True, True]

        self.calculate_syndrome()
        self.error_hat = np.zeros(self.N, dtype=int)
        return self.flooding_decode(L, epsilon)

    def add_error_given_epsilon(self, epsilon):
        self.error = []
        self.errorString = []
        for i in range(self.N):
            rndValue = np.random.rand()
            if rndValue < epsilon / 3:
                self.error.append(1)
                self.errorString.append("X" + str(i))
            elif rndValue < epsilon * 2 / 3:
                self.error.append(2)
                self.errorString.append("Z" + str(i))
            elif rndValue < epsilon:
                self.error.append(3)
                self.errorString.append("Y" + str(i))
            else:
                self.error.append(0)

    def quantize_belief(self, Tau, Tau1, Tau2):
        nom = np.log1p(np.exp(-1.0 * Tau))
        denom = max(-1.0 * Tau1, -1.0 * Tau2) + np.log1p(np.exp(-1.0 * np.fabs((Tau1 - Tau2))))
        ret_val = nom - denom
        if np.isnan(ret_val):
            raise RuntimeError("quantize_belief: Log difference is NaN")
        return ret_val

    def trace_inner_product(self, a, b):
        return not (a == 0 or b == 0 or a == b)

    def flooding_decode(self, L, epsilon):
        success = [False, False]
        L0 = np.log(3.0 * (1 - epsilon) / epsilon)
        lambda0 = np.log((1 + np.exp(-L0)) / (2 * np.exp(-L0)))

        mc2v = np.zeros((self.M, max(self.dc)), dtype=float)
        mv2c = np.zeros((self.N, max(self.dv)), dtype=float)
        Taux = np.zeros(self.N, dtype=float)
        Tauz = np.zeros(self.N, dtype=float)
        Tauy = np.zeros(self.N, dtype=float)
        phi_msg = np.zeros(max(self.dc), dtype=float)

        for decIter in range(L):
            for i in range(self.M):
                phi_sum = 0
                sign_prod = 1.0 if self.syn[i] == 0 else -1.0
                for j in range(self.dc[i]):
                    if mv2c[self.Mc[i][j]][self.Mck[i][j]] != 0.0:
                        phi_msg[j] = -1.0 * np.log(np.tanh(np.fabs(mv2c[self.Mc[i][j]][self.Mck[i][j]]) / 2.0))
                    else:
                        phi_msg[j] = 60
                    phi_sum += phi_msg[j]
                    sign_prod *= 1.0 if mv2c[self.Mc[i][j]][self.Mck[i][j]] >= 0.0 else -1.0

                for j in range(self.dc[i]):
                    phi_extrinsic_phi_sum = phi_sum - phi_msg[j]
                    phi_phi_sum = 60 if phi_extrinsic_phi_sum == 0 else -1.0 * np.log(np.tanh(phi_extrinsic_phi_sum / 2.0))
                    mc2v[i][j] = phi_phi_sum * sign_prod * (1.0 if mv2c[self.Mc[i][j]][self.Mck[i][j]] >= 0.0 else -1.0)
                    if self.mTrained and decIter < self.trained_iter:
                        mc2v[i][j] *= self.weights_cn[decIter][i][j]
                    if np.isnan(mc2v[i][j]):
                        raise RuntimeError("flooding_decode: mc2v[i][j] is NaN")
                    if np.isinf(mc2v[i][j]):
                        raise RuntimeError("flooding_decode: mc2v[i][j] is infinity")

            for Vidx in range(self.N):
                Taux[Vidx] = L0
                Tauz[Vidx] = L0
                Tauy[Vidx] = L0
                if self.mTrained and decIter < self.trained_iter:
                    Taux[Vidx] *= self.weights_llr[decIter + 1][Vidx]
                    Tauy[Vidx] *= self.weights_llr[decIter + 1][Vidx]
                    Tauz[Vidx] *= self.weights_llr[decIter + 1][Vidx]

                for jj in range(self.dv[Vidx]):
                    if self.varVal[Vidx][jj] == 1:
                        Tauz[Vidx] += mc2v[self.Nv[Vidx][jj]][self.Nvk[Vidx][jj]]
                        Tauy[Vidx] += mc2v[self.Nv[Vidx][jj]][self.Nvk[Vidx][jj]]
                    elif self.varVal[Vidx][jj] == 2:
                        Taux[Vidx] += mc2v[self.Nv[Vidx][jj]][self.Nvk[Vidx][jj]]
                        Tauy[Vidx] += mc2v[self.Nv[Vidx][jj]][self.Nvk[Vidx][jj]]
                    elif self.varVal[Vidx][jj] == 3:
                        Taux[Vidx] += mc2v[self.Nv[Vidx][jj]][self.Nvk[Vidx][jj]]
                        Tauz[Vidx] += mc2v[self.Nv[Vidx][jj]][self.Nvk[Vidx][jj]]
                    else:
                        raise ValueError("Something is wrong")

                for jj in range(self.dv[Vidx]):
                    if self.varVal[Vidx][jj] == 1:
                        Tauxi = Taux[Vidx]
                        Tauzi = Tauz[Vidx] - mc2v[self.Nv[Vidx][jj]][self.Nvk[Vidx][jj]]
                        Tauyi = Tauy[Vidx] - mc2v[self.Nv[Vidx][jj]][self.Nvk[Vidx][jj]]
                        temp = self.quantize_belief(Tauxi, Tauyi, Tauzi)
                    elif self.varVal[Vidx][jj] == 2:
                        Tauxi = Taux[Vidx] - mc2v[self.Nv[Vidx][jj]][self.Nvk[Vidx][jj]]
                        Tauzi = Tauz[Vidx]
                        Tauyi = Tauy[Vidx] - mc2v[self.Nv[Vidx][jj]][self.Nvk[Vidx][jj]]
                        temp = self.quantize_belief(Tauzi, Tauyi, Tauxi)
                    elif self.varVal[Vidx][jj] == 3:
                        Tauxi = Taux[Vidx] - mc2v[self.Nv[Vidx][jj]][self.Nvk[Vidx][jj]]
                        Tauzi = Tauz[Vidx] - mc2v[self.Nv[Vidx][jj]][self.Nvk[Vidx][jj]]
                        Tauyi = Tauy[Vidx]
                        temp = self.quantize_belief(Tauyi, Tauxi, Tauzi)
                    else:
                        raise ValueError("Something is wrong")

                    limit = 60
                    if temp > limit:
                        mv2c[Vidx][jj] = limit
                    elif temp < -limit:
                        mv2c[Vidx][jj] = -limit
                    else:
                        mv2c[Vidx][jj] = temp

                    if self.mTrained and decIter < self.trained_iter:
                        mv2c[Vidx][jj] *= self.weights_vn[decIter + 1][Vidx][jj]

                    if np.isnan(mv2c[Vidx][jj]):
                        raise RuntimeError("flooding_decode: mv2c[Vidx][jj] is NaN")
                    if np.isinf(mv2c[Vidx][jj]):
                        raise RuntimeError("flooding_decode: mv2c[Vidx][jj] is infinity")

            success = self.check_success(Taux, Tauy, Tauz)
            if success[0]:
                break

        return success

    def calculate_syndrome(self):
        self.syn = []
        for i in range(self.M):
            check = False
            for j in range(self.dc[i]):
                check = self.trace_inner_product(self.error[self.Mc[i][j]], self.checkVal[i][j])
                check = not check if check else check
            self.syn.append(check % 2)

    def check_success(self, Taux, Tauy, Tauz):
        success = [False, False]
        self.error_hat = [0] * self.N
        for i in range(self.N):
            if Taux[i] > 0 and Tauy[i] > 0 and Tauz[i] > 0:
                self.error_hat[i] = 0
            elif Taux[i] < Tauy[i] and Taux[i] < Tauz[i]:
                self.error_hat[i] = 1
            elif Tauz[i] < Taux[i] and Tauz[i] < Tauy[i]:
                self.error_hat[i] = 2
            else:
                self.error_hat[i] = 3

        for i in range(self.M):
            check = False
            for j in range(self.dc[i]):
                check = self.trace_inner_product(self.error_hat[self.Mc[i][j]], self.checkVal[i][j]) != check
            if check != self.syn[i]:
                return success

        success[0] = True
        for i in range(self.G_rows):
            check = False
            for j in range(self.N):
                check = self.trace_inner_product(self.error[j], self.G[i][j]) != check
                check = self.trace_inner_product(self.error_hat[j], self.G[i][j]) != check
            if check:
                return success

        success[1] = True
        return success


In [134]:
import numpy as np

class ParseResult:
    def __init__(self):
        self.epsilons = []
        self.progname = ""
        self.maximum_frame_errors = 30
        self.maximum_decoded_words = 45000000
        self.print_help = False

def print_help(progname):
    print(
        "Neural Belief Propagation Using Overcomplete Check Matrices\n"
        "\n"
        "Usage:\n"
        f"{progname} [options] [Epsilon] [Epsilon]...\n"
        "\n"
        "Positional arguments:\n"
        "  Epsilon                     optionally specify a list of epsilons at which to verify.\n"
        "                              If no epsilons are given, use default list of epsilons \n"
        "\n"
        "Options:\n"
        "  -r|--range START STEP STOP  Specify a decreasing range of epsilons to use, e.g.,\n"
        "                              \"-r 0.1 0.025 0.9\" to get 0.1 0.975 0.95 0.925 0.9\n"
        "\n"
        "  -e|--max-frame-errors N     Stop simulating after observing N frame errors\n"
        "  -w|--max-decoded-words N    Stop simulating after decoding N words\n"
        "\n\n\n"
        " This program accompanies the paper\n"
        "     S. Miao, A. Schnerring, H. Li and L. Schmalen,\n"
        "     \"Neural belief propagation decoding of quantum LDPC codes using overcomplete check matrices,\"\n"
        "     Proc. IEEE Inform. Theory Workshop (ITW), Saint-Malo, France, Apr. 2023, "
        "https://arxiv.org/abs/2212.10245\n"
    )

def parse_arguments(argument_count, arguments, default_epsilons, maximum_frame_errors=30, maximum_decoded_words=45000000):
    result = ParseResult()
    result.maximum_frame_errors = maximum_frame_errors
    result.maximum_decoded_words = maximum_decoded_words
    if arguments:  # Check if arguments list is not empty
        result.progname = arguments[0]
        arguments = arguments[1:]
        argument_count -= 1
    else:
        result.progname = "default_progname"
    
    
    

    if not argument_count:  # no arguments passed – use default epsilons
        result.epsilons = default_epsilons
        return result  # return empty list

    while argument_count:
        argument = arguments[0]
        argument_view = argument

        if argument_view in ("-h", "--help"):
            result.print_help = True
            return result
        if argument_view in ("-r", "--range"):
            if argument_count < 4:  # need three more arguments
                result.print_help = True
                return result
            start = float(arguments[1])
            step = float(arguments[2])
            stop = float(arguments[3])
            argument_count -= 3
            try:
                if stop <= 0.0:
                    raise ValueError("Stop value needs to be positive")
                if step <= 0.0:
                    raise ValueError("step value needs to be positive")
                if start <= stop:
                    raise ValueError("Stop value needs to be smaller than start value")
                result.epsilons = list(np.arange(start, stop, step)[::-1])
            except ValueError as exceptions:
                print(f"Could not parse range: {exceptions}; skipping range.")
        elif argument_view in ("-e", "--max-frame-errors"):
            argument_count -= 1
            result.maximum_frame_errors = int(arguments[1])
        elif argument_view in ("-w", "--max-decoded-words"):
            argument_count -= 1
            result.maximum_decoded_words = int(arguments[1])
        else:
            # we encountered something that doesn't look like a keyword argument -- proceed to parse numbers
            break
        arguments = arguments[1:]
        argument_count -= 1

    if not argument_count and not result.epsilons:  # no arguments passed, and no range found – use default epsilons
        result.epsilons = default_epsilons
        return result

    result.epsilons = []
    for argument in arguments:
        value = float(argument)
        if value <= 0.0:  # couldn't parse number, or actually 0.0 (or below) was passed
            print(f"Ignoring argument '{argument}' as unparseable or non-positive.")
            continue  # skip this
        result.epsilons.append(value)
    return result

In [135]:
n = 32
k = 2
m = 96
trained_iter=18
decIterNum = 18
trained = True
ep0 = 0.37
codeType = "toric"

matrix_supplier = FileReader(n, k, m, codeType, trained)



Length of dv: 32
Length of dc: 96
Length of Nvk: 32
Length of Mck: 96
Length of Nv: 32
Length of Mc: 96
Length of checkVal: 96
Length of varVal: 32
32
[[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [136]:
matrix_supplier.check_symplectic()

ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
1 0 0 0 ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
0 1 0 0 ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
0 0 1 0 ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
0 0 0 1 ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
0 0 0 0 ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao
ciao


True

In [123]:

import helpers

def main():
    n = 32
    k = 2
    m = 96
    trained_iter=18
    decIterNum = 18
    trained = True
    ep0 = 0.37
    codeType = "toric"
    print("PROVA1")
    matrix_supplier = FileReader(n, k, m, codeType, trained)
    matrix_supplier.save_values_to_file()
    matrix_supplier.check_symplectic()

    default_max_frame_errors = 300
    default_max_decoded_words = 45000000
    default_ep_list = [
        0.15, 0.135, 0.12, 0.105, 0.09, 0.075, 0.06, 0.045, 0.03, 0.015
    ]

    arguments = parse_arguments(argument_count=0, arguments=[], default_epsilons=default_ep_list,
                                    maximum_frame_errors=default_max_frame_errors,
                                    maximum_decoded_words=default_max_decoded_words)


    ep_list = arguments.epsilons
    max_frame_errors = arguments.maximum_frame_errors
    max_decoded_words = arguments.maximum_decoded_words

    print(f"% [[{n},{k}]], {m} checks, {decIterNum} iter ", end="")
    if trained:
        print(",trained")
    print("\n% collect", max_frame_errors, "frame errors or", max_decoded_words, "decoded error patterns")

    total_decoding = 0
    failure = 0

    for epsilon in ep_list:
        while failure <= max_frame_errors and total_decoding <= max_decoded_words:
            code = StabilizerCodes(n, k, m, codeType, matrix_supplier, trained)
            code.add_error_given_epsilon(epsilon)
            success = code.decode(decIterNum, ep0)
            
            if not success[1]:
                failure += 1
            total_decoding += 1

        print("% FE", failure, ", total dec.", total_decoding, "\\\\")
        print(epsilon, failure / total_decoding, "\\\\")
        total_decoding = 0
        failure = 0

    while True:
        pass

if __name__ == "__main__":
    main()

PROVA1
Length of dv: 32
Length of dc: 96
Length of Nvk: 32
Length of Mck: 96
Length of Nv: 32
Length of Mc: 96
Length of checkVal: 96
Length of varVal: 32
32
[[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,

KeyboardInterrupt: 

In [ ]:
print(matrix_supplier.compare_Mck_with_file("Mck.txt"))
print(matrix_supplier.compare_Mc_with_file("Mc.txt"))
 print(matrix_supplier.compare_checkVal_with_file("checkVal.txt"))
    print(matrix_supplier.compare_varVal_with_file("varVal.txt"))
    print(matrix_supplier.compare_Nv_with_file("Nv.txt"))
    print(matrix_supplier.compare_Nvk_with_file("Nvk.txt"))

In [ ]:
print("maxDv:", self.maxDv)
print("maxDc:", self.maxDc)
print("dv:", self.dv)
print("dc:", self.dc)
print("Nvk:", self.Nvk)
print("Mck:", self.Mck)
print("Nv:", self.Nv)
print("Mc:", self.Mc)
print("checkVal:", self.checkVal)
 print("varVal:", self.varVal)